Exactly like addsenseboarddata from Straight Run:
# addsenseboarddata – Enrich `all_data.csv` with SenseBoard Loadcell Signals

This notebook augments the unified dataset (`all_data.csv`) with high-frequency **SenseBoard** measurements (loadcells, moments, and positions) by time-aligning SenseBoard logs to SenseBoard boat rows.

## What this notebook does

1. **Load base dataset**
   - Reads `all_data.csv`, parses `ISODateTimeUTC`, and tags each row with an internal `__original_id__` to preserve identity during merges.

2. **Split SenseBoard vs. other boats**
   - Separates rows where `boat_name == "SenseBoard"` from all other boats.  
   - Only SenseBoard rows are candidates for enrichment with loadcell data.

3. **Scan SenseBoard logs**
   - For each session folder under `../Data_Sailnjord/Straight_lines/<DATE>/senseboard_log/`, loads the Excel file `SenseBoard_log_modified*.xlsx`.
   - Keeps relevant columns when present:  
     `Timestamp GPS`, `Latitude`, `Longitude`, `Euler_X (deg)`, `Euler_Y (deg)`,  
     `LoadCell_1..6`, `F_front`, `F_back`, `M_tot_X`, `M_tot_Y`,  
     `M_front_X`, `M_front_Y`, `M_back_X`, `M_back_Y`, `P_front_X`, `P_front_Y`, `P_back_X`, `P_back_Y`.

4. **Clean and normalize SenseBoard timestamps**
   - Parses `Timestamp GPS` with microsecond precision and localizes to UTC.
   - Converts numeric columns; drops invalid timestamps.
   - **Spreads duplicate timestamps** using `spread_duplicate_timestamps(...)` to micro-jitter duplicates (default 100 ms spacing) so that as-of merges are well-defined.

5. **Time alignment (`merge_asof`)**
   - For SenseBoard rows whose `ISODateTimeUTC` falls within the log’s time range, performs nearest-neighbor alignment:
     - `left_on = ISODateTimeUTC` (from `all_data.csv`)
     - `right_on = Timestamp GPS` (from SenseBoard log)
     - `tolerance = 200 ms`, `direction = 'nearest'`
   - Collects enriched chunks across all sessions.

6. **Post-merge integrity checks**
   - Drops duplicate `__original_id__` after enrichment to keep a single enriched record per original row.
   - Recombines enriched SenseBoard rows with untouched non-SenseBoard rows.
   - Restores any missing mirrored rows (same boat/opponent/timestamp) by comparing against the expected pairs from the original dataset and re-inserting any lost entries.

7. **Output**
   - Writes the final, time-aligned dataset to **`all_data_enriched.csv`** (drops `__original_id__`).

## Notes
- The enrichment is **non-destructive**: if no SenseBoard log matches a time span, original rows are preserved unmodified.
- The 200 ms tolerance and 100 ms de-duplication spacing are configurable if your logs’ timing characteristics change.


In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import openpyxl

def spread_duplicate_timestamps(df, timestamp_col="Timestamp GPS", interval_ms=100):
    new_rows = []
    for ts, group in df.groupby(timestamp_col):
        if len(group) == 1:
            new_rows.append(group)
        else:
            deltas = pd.to_timedelta(np.linspace(0, interval_ms, len(group), endpoint=False), unit="ms")
            group = group.copy()
            group[timestamp_col] = ts + deltas
            new_rows.append(group)
    return pd.concat(new_rows).sort_values(timestamp_col).reset_index(drop=True)

# === Step 1: Load data ===
all_data = pd.read_csv("all_data.csv")
all_data["ISODateTimeUTC"] = pd.to_datetime(all_data["ISODateTimeUTC"], errors="coerce")
all_data["__original_id__"] = all_data.index  # Track original

# === Step 2: Split rows ===
senseboard_rows = all_data[all_data["boat_name"] == "SenseBoard"].copy()
other_rows = all_data[all_data["boat_name"] != "SenseBoard"].copy()

# === Step 3: Prepare enrichment ===
root_dir = Path("../Data_Sailnjord/Maneuvers/")  # Adjust path as necessary
merged_parts = []

cols_to_keep = [
    "Timestamp GPS", "Latitude", "Longitude", "Euler_X (deg)", "Euler_Y (deg)",
    "LoadCell_1", "LoadCell_2", "LoadCell_3", "LoadCell_4", "LoadCell_5", "LoadCell_6",
    "F_front", "F_back",
    "M_tot_X", "M_tot_Y",
    "M_front_X", "M_front_Y",
    "M_back_X", "M_back_Y",
    "P_front_X", "P_front_Y",
    "P_back_X", "P_back_Y"
]

for subfolder in root_dir.iterdir():
    if not subfolder.is_dir():
        continue

    print(f"Processing folder: {subfolder.name}")
    log_dir = subfolder / "senseboard_log"
    senseboard_file = next(log_dir.glob("SenseBoard_log_modified*.xlsx"), None) if log_dir.exists() else None
    if senseboard_file is None:
        print(f"No SenseBoard file in {subfolder.name}")
        continue

    sb_data = pd.read_excel(senseboard_file, dtype=str, engine="openpyxl")
    sb_data.columns = sb_data.columns.str.strip()

    if "Timestamp GPS" not in sb_data.columns:
        raise ValueError("Missing 'Timestamp GPS'")

    sb_data["Timestamp GPS"] = pd.to_datetime(sb_data["Timestamp GPS"], format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")
    sb_data = sb_data.dropna(subset=["Timestamp GPS"])
    sb_data["Timestamp GPS"] = sb_data["Timestamp GPS"].dt.tz_localize("UTC")

    for col in sb_data.columns:
        if col != "Timestamp GPS":
            sb_data[col] = pd.to_numeric(sb_data[col], errors="coerce")

    sb_data = spread_duplicate_timestamps(sb_data, interval_ms=100)
    sb_data = sb_data[[col for col in cols_to_keep if col in sb_data.columns]]

    start_time, end_time = sb_data["Timestamp GPS"].min(), sb_data["Timestamp GPS"].max()

    sb_rows_sub = senseboard_rows[
        (senseboard_rows["ISODateTimeUTC"] >= start_time) &
        (senseboard_rows["ISODateTimeUTC"] <= end_time)
    ].copy()

    if sb_rows_sub.empty:
        print(f"No matching SenseBoard rows in {subfolder.name}")
        continue

    merged = pd.merge_asof(
        sb_rows_sub.sort_values("ISODateTimeUTC"),
        sb_data.sort_values("Timestamp GPS"),
        left_on="ISODateTimeUTC",
        right_on="Timestamp GPS",
        direction="nearest",
        tolerance=pd.Timedelta("200ms")  # Increased from 50ms
    )

    merged_parts.append(merged)

# === Step 4: Merge enriched data ===
if merged_parts:
    full_merged = pd.concat(merged_parts, ignore_index=True)
else:
    full_merged = senseboard_rows.copy()

full_merged = full_merged.drop_duplicates(subset=["__original_id__"])

# === Step 5: Combine with untouched rows ===
final = pd.concat([full_merged, other_rows], ignore_index=True)
final = final.sort_values("ISODateTimeUTC")

# === Step 6: Restore missing mirrored rows ===
expected_pairs = all_data[["boat_name", "ISODateTimeUTC", "__original_id__"]]
actual_pairs = final[["boat_name", "ISODateTimeUTC"]].copy()

merged_check = pd.merge(
    expected_pairs,
    actual_pairs,
    on=["boat_name", "ISODateTimeUTC"],
    how="left",
    indicator=True
)

missing = merged_check.query("_merge == 'left_only'")
recovered = pd.merge(
    missing[["__original_id__"]],
    all_data,
    on="__original_id__",
    how="left"
)

final = pd.concat([final, recovered], ignore_index=True).sort_values("ISODateTimeUTC")

# === Step 7: Save output ===
final.drop(columns="__original_id__", errors="ignore").to_csv("all_data_enriched.csv", index=False)
print("✅ Enriched dataset saved to all_data_enriched.csv")


Processing folder: 08_06


Processing folder: 11_06


✅ Enriched dataset saved to all_data_enriched.csv


In [2]:
print(len(final))
print(len(all_data))

35314
35314
